# Проработка подходов по увеличению выручки интернет-магазина <a class="tocSkip">

**Цель исследования:**  
Приоритизировать гипотезы для увеличения выручки интернет-магазина, запустить A/B-тест и проанализировать результаты.   

## Загрузка данных и подготовка их к анализу

In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np
import scipy.stats as stats
import datetime as dt
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# загрузка данных
try:
    hypothesis, orders, visitors = (
        pd.read_csv('/datasets/hypothesis.csv'),  
        pd.read_csv('/datasets/orders.csv'),
        pd.read_csv('/datasets/visitors.csv')
    ) 
except FileNotFoundError:
    print('Неверно указан путь к файлу.')
except:
    print('Ошибка.')

In [ ]:
display(hypothesis, orders, visitors)

- Описание данных  
  
Таблица hypothesis содержит 9 гипотез по увеличению выручки интернет-магазина с указанными параметрами Reach, Impact, Confidence, Effort:  

    Hypothesis — краткое описание гипотезы;  
    Reach — охват пользователей по 10-балльной шкале;
    Impact — влияние на пользователей по 10-балльной шкале;
    Confidence — уверенность в гипотезе по 10-балльной шкале;
    Efforts — затраты ресурсов на проверку гипотезы по 10-балльной шкале.
    
Таблица orders содержит информацию о заказах, совершенных пользователями:  

    transactionId — идентификатор заказа;
    visitorId — идентификатор пользователя, совершившего заказ;
    date — дата, когда был совершён заказ;
    revenue — выручка заказа;
    group — группа A/B-теста, в которую попал заказ.    
    
Таблица visitors содержит информацию о количестве пользователей:  

    date — дата;
    group — группа A/B-теста;
    visitors — количество пользователей в указанную дату в указанной группе A/B-теста.    

Мы видим, что столбцы таблиц hypothesis и orders записаны в разном регистре. Для соблюдения единообразия изменим названия колонок в таблицах. 

In [ ]:
# переименование колонок
hypothesis.columns = ['hypothesis', 'reach', 'impact', 'confidence','efforts']
orders.columns = ['transaction_id', 'visitor_id', 'date', 'revenue', 'group']

Посмотрим на общую информацию о таблицах.

In [ ]:
# список таблиц
list_of_data = [hypothesis, orders, visitors]

In [ ]:
# общая информация о таблицах
for data in list_of_data:
    display(data.info()) 

Пропуски в таблицах отсутствуют. Однако колонки с датами имеют тип данных object. Для дальнейшей работы с датой необходимо их преобразовать в тип datetime.

In [ ]:
# преобразование данных о датах
orders['date'] = orders['date'].map(
    lambda x: dt.datetime.strptime(x, '%Y-%m-%d')
)
visitors['date'] = visitors['date'].map(
    lambda x: dt.datetime.strptime(x, '%Y-%m-%d')
)

Проверим таблицы на наличие дубликатов.

In [ ]:
# проверка на дубликаты
for data in list_of_data:
    print('Количество дубликатов:', data.duplicated().sum())

Проверим не попали ли одни и теже пользователи в обе группы тестирования. 

In [ ]:
# выберем id пользователей группы А
visitors_a = orders.query('group == "A"')['visitor_id']

# проверим есть ли id пользователей группы А в группе В
users_both_groups = orders.query('group == "B" and visitor_id in @visitors_a')['visitor_id']
print('Количество пользователей, попавших в обе группы: ', len(users_both_groups.drop_duplicates()))

Мы выявили 58 пользователей, которые попали в обе группы тестирования. Таких пользователей нужно выявлять на этапе А/А-тестирования. В данном случае по итогам А/В-тестирования мы не cможем корректно их удалить, так как они сагрегированны в таблице `visitors`.

### Вывод

Мы единообразно изменили названия колонок, привели данные к нужному типу и проверили на наличие пропусков и дубликатов. Теперь данные готовы к дальнейшему анализу.

## Часть 1. Приоритизация гипотез

### Приоритизация гипотез с помощью фреймворка ICE 

Применим фреймворк ICE для приоритизации гипотез и отсортируем их по убыванию приоритета.

In [ ]:
hypothesis['ice'] = ((hypothesis['impact'] * hypothesis['confidence']) / hypothesis['efforts']).round(2)
pd.set_option('max_colwidth', 120)
hypothesis[['hypothesis', 'ice']].sort_values('ice', ascending=False)

### Приоритизация гипотез с помощью фреймворка RICE 

Применим фреймворк RICE для приоритизации гипотез и отсортируем их по убыванию приоритета.

In [ ]:
hypothesis['rice'] = (hypothesis['reach'] * hypothesis['impact'] * hypothesis['confidence']) / hypothesis['efforts']

hypothesis[['hypothesis', 'rice']].sort_values('rice', ascending=False)

### Отличия в приоритизации гипотез с помощью фреймворков ICE и RICE 

Сделаем ранжировку гипотез, где самым приоритетным гипотезам будет соответствовать 9 баллов, а наименее приоритетным 1 балл. Также визуализируем данные для наглядности.

In [ ]:
# ранжирование гипотез
hypothesis_rice = hypothesis[['hypothesis', 'rice']].sort_values('rice').reset_index()
for i in range(0, 9):
     hypothesis_rice.loc[i, 'priority_rice'] = (i+1)
        
hypothesis_ice = hypothesis[['hypothesis', 'ice']].sort_values('ice').reset_index()
for i in range(0, 9):
     hypothesis_ice.loc[i, 'priority_ice'] = (i+1)
        
# объединение таблиц        
total_hypothesis = (hypothesis_ice
                    .merge(hypothesis_rice, on='hypothesis')
                    .drop(['index_y'], axis=1)
                    .sort_values('priority_rice', ascending=False)
                   )
total_hypothesis.columns = ['num_of_hypothesis', 'hypothesis', 'ice', 'priority_ice', 'rice', 'priority_rice']
total_hypothesis['num_of_hypothesis'] = total_hypothesis['num_of_hypothesis'].astype('object')
total_hypothesis

In [ ]:
# визуализация 
cm = sns.light_palette("green", as_cmap=True)

(total_hypothesis[['num_of_hypothesis', 'hypothesis', 'priority_ice', 'priority_rice']] 
 .style
 .background_gradient(cmap=cm)
)

При применении фреймворка ICE самой приоритетной оказалась гипотеза № 8, а вот при применении фреймворка RICE - гипотеза № 7. Отличие в том, что фреймворк RICE в отличие от ICE учитывает охват пользователей. Гипотеза № 7 имеет 10 баллов из 10 по графе "*reach*" - охват пользователей. Наименее приоритетной при использовании обоих фреймворков является гипотеза № 4.  

## Часть 2. Анализ A/B-теста

### График кумулятивной выручки по группам

Чтобы построить графики, собирем кумулятивные данные. Создадим массив уникальных пар значений дат и групп теста.

In [ ]:
# массив уникальных пар значений дат и групп теста
datesGroups = orders[['date','group']].drop_duplicates() 
datesGroups.head()

Соберём агрегированные  по дням кумулятивные данные о заказах.

In [ ]:
# агрегированные по дням кумулятивные данные о покупках
orders_aggregated = datesGroups.apply(
    lambda x: orders[
        np.logical_and(
            orders['date'] <= x['date'], orders['group'] == x['group']
        )
    ].agg(
        {
            'date': 'max',
            'group': 'max',
            'transaction_id': 'nunique',
            'visitor_id': 'nunique',
            'revenue': 'sum',
        }
    ),
    axis=1,
).sort_values(by=['date', 'group'])

# переименование колонок
orders_aggregated.columns = ['date', 'group', 'orders', 'buyers', 'revenue']
orders_aggregated.head()

Построим графики кумулятивной выручки по дням и группам A/B-тестирования:

In [ ]:
# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulative_revenue_a = orders_aggregated[orders_aggregated['group']=='A'][['date','revenue', 'orders']]

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulative_revenue_b = orders_aggregated[orders_aggregated['group']=='B'][['date','revenue', 'orders']]

plt.figure(figsize=(11,7))

# график выручки группы А
plt.plot(cumulative_revenue_a['date'], cumulative_revenue_a['revenue'], label='A')

# график выручки группы B
plt.plot(cumulative_revenue_b['date'], cumulative_revenue_b['revenue'], label='B')

plt.legend()
plt.title('График кумулятивной выручки по группам A/B-тестирования', fontsize=15)
plt.xlabel('Даты', fontsize=12) 
plt.ylabel('Выручка', fontsize=12)
plt.show()

Выручка увеличивается в течение всего теста. Однако график выручки группы В резко вырос 19-20 августа 2019 г. Это может быть всплеском числа заказов, либо появлением очень дорогих заказов в выборке, то есть выбросов.

### График кумулятивного среднего чека по группам

Построим графики среднего чека по группам А/В-тестирования.

In [ ]:
plt.figure(figsize=(11,7))
# график среднего группы А
plt.plot(cumulative_revenue_a['date'], cumulative_revenue_a['revenue']/cumulative_revenue_a['orders'], label='A')

# график среднего группы В
plt.plot(cumulative_revenue_b['date'], cumulative_revenue_b['revenue']/cumulative_revenue_b['orders'], label='B')
plt.legend() 
plt.title('График кумулятивного среднего чека по группам A/B-тестирования', fontsize=15)
plt.xlabel('Даты', fontsize=12) 
plt.ylabel('Средний чек', fontsize=12)
plt.show()

Средний чек установился ближе к концу теста для группы А, но начал падать для группы B. Вероятно, в группу B начиная с 19-20 августа попали крупные заказы (резкий всплеск на графике). Тогда ей нужно больше данных, чтобы прийти к реальному среднему чеку и установиться на его уровне.

### График относительного изменения кумулятивного среднего чека группы B к группе A

Построим график относительного различия для среднего чека. 

In [ ]:
# собираем данные в одном датафрейме
merged_cumulative_revenue = (
    cumulative_revenue_a
    .merge(cumulative_revenue_b, left_on='date', right_on='date', how='left', suffixes=['_a', '_b'])
)

# график отношения средних чеков
plt.figure(figsize=(11,7))
(
plt.plot(merged_cumulative_revenue['date'], (merged_cumulative_revenue['revenue_b']/merged_cumulative_revenue['orders_b'])\
                                          /(merged_cumulative_revenue['revenue_a']/merged_cumulative_revenue['orders_a'])-1)
)

# линия уровня
plt.axhline(y=0, color='red', linestyle='--') 

plt.title('График относительного изменения кумулятивного среднего чека группы B к группе A', fontsize=14)
plt.xlabel('Даты', fontsize=12) 
plt.show()

В целом средний чек группы В значительно превышает среднйи чек группы А. Однако в нескольких точках график различия между сегментами резко «скачет». Вероятно это связано с крупными заказами и выбросами. 

### График кумулятивной конверсии по группам

Построим график кумулятивной конверсии по группам А/В-тестирования. Для этого сначала соберем кумулятивные данные о посетителях интернет-магазина по дням. Затем поисчтаем кумулятивную конверсию по группам. 

In [ ]:
# агрегированные кумулятивные по дням данные о посетителях 
visitors_aggregated = datesGroups.apply(
    lambda x: visitors[
        np.logical_and(
            visitors['date'] <= x['date'], visitors['group'] == x['group']
        )
    ].agg({'date': 'max', 'group': 'max', 'visitors': 'sum'}),
    axis=1,
).sort_values(by=['date', 'group'])

# объединение кумулятивных данных в одну таблицу 
cumulative_data = orders_aggregated.merge(
    visitors_aggregated, left_on=['date', 'group'], right_on=['date', 'group']
)

# переименование столбцов
cumulative_data.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']
cumulative_data.head()

In [ ]:
# кумулятивная конверсия
cumulative_data['conversion'] = cumulative_data['orders']/cumulative_data['visitors']

# данные по группе A
cumulative_data_a = cumulative_data[cumulative_data['group']=='A']

# данные по группе B
cumulative_data_b = cumulative_data[cumulative_data['group']=='B']

# графики
plt.figure(figsize=(11,6))
plt.plot(cumulative_data_a['date'], cumulative_data_a['conversion'], label='A')
plt.plot(cumulative_data_b['date'], cumulative_data_b['conversion'], label='B')
plt.legend()
plt.title('График кумулятивной конверсии по группам A/B-тестирования', fontsize=15)
plt.xlabel('Даты', fontsize=12) 
plt.ylabel('Конверсия', fontsize=12)

# масштаб осей
plt.axis([dt.datetime.strptime('2019-08-01', '%Y-%m-%d'), dt.datetime.strptime('2019-09-01', '%Y-%m-%d'), 0, 0.040])
plt.show()

Через неделю после начала тестирования конверсия группы В вырвалась вперёд и зафиксировалась на уровне около 3,4%, а конверсия группы А просела и также зафиксировалась на уровне 3%.

### График относительного изменения кумулятивной конверсии группы B к группе A

Построим график относительного различия кумулятивных конверсий.

In [ ]:
# собираем данные в одном датафрейме
merged_cumulative_conversions = (
    cumulative_data_a[['date','conversion']]
    .merge(cumulative_data_b[['date','conversion']], left_on='date', right_on='date', how='left', suffixes=['A', 'B'])
)

# график отношения конверсии
plt.figure(figsize=(11,6))
plt.plot(
    merged_cumulative_conversions['date'], 
    merged_cumulative_conversions['conversionB']/merged_cumulative_conversions['conversionA']-1, 
    label="Относительный прирост конверсии группы B к группе A"    
)
plt.legend()

plt.axhline(y=0, color='red', linestyle='--')
plt.axhline(y=0.1, color='grey', linestyle='--')
plt.axis(['2019-08-01', '2019-09-01', -0.5, 0.5]) 
plt.title('График относительного изменения конверсии группы B к группе А', fontsize=15)
plt.xlabel('Даты', fontsize=12) 
plt.show()

В первую неделю теста группа B проигрывала группе A, затем вырвалась вперёд. Менее, чем через 3 недели после начала теста её конверсия начала снижаться.
В целом можно сказать, что конверсия уже установилась и присрост конверсии группы В отсносительно группы А составляет 10-14%. Впрочем, сперва стоит проанализировать аномалии, возможно, они изменят картину. 

### Точечный график количества заказов по пользователям

Построим точечную диаграмму числа заказов на одного пользователя.

In [ ]:
# количество заказов по пользователям 
orders_by_users = (
    orders.groupby('visitor_id', as_index=False)
    .agg({'transaction_id': 'nunique'})
)

# переименование столбцов
orders_by_users.columns = ['user_id', 'orders']

# точечный график
x_values = pd.Series(range(0,len(orders_by_users)))
plt.figure(figsize=(9,5))
plt.scatter(x_values, orders_by_users['orders']) 
plt.title('График количества заказов по пользователям', fontsize=15)
plt.xlabel('Пользователи', fontsize=12) 
plt.ylabel('Заказы', fontsize=12) 
plt.show()

Большинство покупателей делают только один заказ. Много пользователей с 2-4 заказами. Чтобы понять считать их аномалиями или нет, воспользумся подсчетом перцентилей.

###  Определение границы аномальных пользователей

Посчитаем 95-й и 99-й перцентили количества заказов на пользователя и выберем границу для определения аномальных пользователей.

In [ ]:
np.percentile(orders_by_users['orders'], [95, 99])

95% прцентов пользователей совершают 1-2 заказа. Примем за аномальных пользователей тех, кто совершил больше 2 заказов. Так мы уберём 5% пользователей с большим числом заказов.

### Точечный график стоимостей заказов

Посторим точечный график стоимостей заказов, чтобы увидеть аномально дорогие заказы.

In [ ]:
# точечный график
x_values = pd.Series(range(0,len(orders['revenue'])))
plt.figure(figsize=(9,5))
plt.scatter(x_values, orders['revenue']) 
plt.title('График стоимостей заказов', fontsize=15)
plt.xlabel('Заказы', fontsize=12) 
plt.ylabel('Стоимость заказа', fontsize=12) 
plt.show()

Как мы и предполагали есть несколько дорогих заказов, которые доходят до 1,2 млн. рублей. По графику можно выбрать границу аномальных заказов около 100 000 рублей. Однако для определения точной границы воспользумся подсчетом перцентилей. 

### Определение границы аномальных заказов

Посчитаем 95-й и 99-й перцентили стоимости заказов и выберем границу для определения аномальных заказов.

In [ ]:
np.percentile(orders['revenue'], [95, 99])

5% пользователей совершают заказы на сумму более 28 000 рублей, а 1% пользователей на сумму более 58 000 тыясч рублей. Примем за аномальных пользователей тех, кто совершил заказ дороже 30 000 рублей. Так мы уберём от 1% до 5% пользователей с дорогими заказами.

### Cтатистическая значимость различий в конверсии между группами по «сырым» данным

Посчитаем статистическую значимость различий в конверсии между группами А/В-тестирования по «сырым» данным. Для этого сначала посчитаем количество пользователей, количество закаов и выручку по группам А/В-тестирования в каждую дату и с нарастающим итогом. Затем объединим все данные в одну талицу. 

In [ ]:
# количество пользователей в выбранную дату в группе А
visitors_a_daily = visitors[visitors['group'] == 'A'][['date', 'visitors']]
visitors_a_daily.columns = ['date', 'visitors_per_date_a']

# количество пользователей до выбранной даты вкл. в группе A
visitors_a_cummulative = visitors_a_daily.apply(
    lambda x: visitors_a_daily[visitors_a_daily['date'] <= x['date']].agg(
        {'date': 'max', 'visitors_per_date_a': 'sum'}
    ),
    axis=1,
)
visitors_a_cummulative.columns = ['date', 'visitors_cummulative_a']

# количество пользователей в выбранную дату в группе В
visitors_b_daily = visitors[visitors['group'] == 'B'][['date', 'visitors']]
visitors_b_daily.columns = ['date', 'visitors_per_date_b']

# количество пользователей до выбранной даты вкл. в группе В
visitors_b_cummulative = visitors_b_daily.apply(
    lambda x: visitors_b_daily[visitors_b_daily['date'] <= x['date']].agg(
        {'date': 'max', 'visitors_per_date_b': 'sum'}
    ),
    axis=1,
)
visitors_b_cummulative.columns = ['date', 'visitors_cummulative_b']

# количество заказов в выбранную дату в группе A
# суммарная выручка в выбранную дату в группе A
orders_a_daily = (
    orders[orders['group'] == 'A'][['date', 'transaction_id', 'visitor_id', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transaction_id': pd.Series.nunique, 'revenue': 'sum'})
)
orders_a_daily.columns = ['date', 'orders_per_date_a', 'revenue_per_date_a']

# суммарное число заказов до выбранной даты вкл. в группе A
# суммарная выручка до выбранной даты вкл. в группе А
orders_a_cummulative = orders_a_daily.apply(
    lambda x: orders_a_daily[orders_a_daily['date'] <= x['date']].agg(
        {'date': 'max', 'orders_per_date_a': 'sum', 'revenue_per_date_a': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])
orders_a_cummulative.columns = [
    'date',
    'orders_cummulative_a',
    'revenue_cummulative_a',
]

# количество заказов в выбранную дату в группе В
# суммарная выручка в выбранную дату в группе В
orders_b_daily = (
    orders[orders['group'] == 'B'][['date', 'transaction_id', 'visitor_id', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transaction_id': pd.Series.nunique, 'revenue': 'sum'})
)
orders_b_daily.columns = ['date', 'orders_per_date_b', 'revenue_per_date_b']

# суммарное число заказов до выбранной даты вкл. в группе В
# суммарная выручка до выбранной даты вкл. в группе В
orders_b_cummulative = orders_b_daily.apply(
    lambda x: orders_b_daily[orders_b_daily['date'] <= x['date']].agg(
        {'date': 'max', 'orders_per_date_b': 'sum', 'revenue_per_date_b': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])
orders_b_cummulative.columns = [
    'date',
    'orders_cummulative_b',
    'revenue_cummulative_b',
]

# объединение таблиц
total_data = (
    orders_a_daily.merge(
        orders_b_daily, left_on='date', right_on='date', how='left'
    )
    .merge(orders_a_cummulative, left_on='date', right_on='date', how='left')
    .merge(orders_b_cummulative, left_on='date', right_on='date', how='left')
    .merge(visitors_a_daily, left_on='date', right_on='date', how='left')
    .merge(visitors_b_daily, left_on='date', right_on='date', how='left')
    .merge(visitors_a_cummulative, left_on='date', right_on='date', how='left')
    .merge(visitors_b_cummulative, left_on='date', right_on='date', how='left')
)

total_data.head() 

Создадим переменные orders_by_users_a и orders_by_users_b со столбцами ['user_id', 'orders']. В них для пользователей, которые заказывали хотя бы 1 раз, укажем число совершённых заказов.

In [ ]:
orders_by_users_a = (
    orders[orders['group'] == 'A']
    .groupby('visitor_id', as_index=False)
    .agg({'transaction_id': 'nunique'})
)
orders_by_users_a.columns = ['visitor_id', 'orders']

orders_by_users_b = (
    orders[orders['group'] == 'B']
    .groupby('visitor_id', as_index=False)
    .agg({'transaction_id': 'nunique'})
)
orders_by_users_b.columns = ['visitor_id', 'orders'] 

Объявим переменные sample_a и sample_b, в которых пользователям из разных групп будет соответствовать количество заказов. Тем, кто ничего не заказал, будут соответствовать нули.

In [ ]:
sample_a = pd.concat(
    [
        orders_by_users_a['orders'],
        pd.Series(
            0,
            index=np.arange(
                total_data['visitors_per_date_a'].sum() - len(orders_by_users_a['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

sample_b = pd.concat(
    [
        orders_by_users_b['orders'], 
        pd.Series(
            0, 
            index=np.arange(
                total_data['visitors_per_date_b'].sum() - len(orders_by_users_b['orders'])
            ), 
            name='orders'
        )
    ], 
    axis=0
) 

- Для того, чтобы определить каким критерием проверять статистическую значимость различий данных в выборках: параметрическим или непараметрическим, проверим данные на нормальное распределение. Для это используем критерий Шапиро-Уилка.

Сформулируем нулевую и альтернативную гипотезы:    
    
**H0** - количество заказов, совершаемых пользователями распределено нормально    
**H1** - распределение не нормально
  
Установим уровень статистической значимости α = 0,05.

In [ ]:
alpha = 0.05  

results = stats.shapiro(sample_a)
p_value = results[1]  

print('p-значение: ', p_value)

if p_value < alpha:
    print('Отвергаем нулевую гипотезу: распределение не нормально')
else:
    print('Не получилось отвергнуть нулевую гипотезу, всё нормально') 

In [ ]:
alpha = 0.05  

results = stats.shapiro(sample_b)
p_value = results[1]  

print('p-значение: ', p_value)

if p_value < alpha:
    print('Отвергаем нулевую гипотезу: распределение не нормально')
else:
    print('Не получилось отвергнуть нулевую гипотезу, всё нормально') 

По итогам проверки выборок мы видим, что данные распределены не нормально, а значит для выявления различий нам стоит применить непараметрический критерий Манна-Уитни. 

Сформулируем нулевую и альтернативную гипотезы:    
    
**H0** - статистически значимых различий в конверсии между группами А/В-тестирования нет    
**H1** - конверсии между группами А/В-тестирования имеют статистически значимые отличия
  
Установим уровень статистической значимости α = 0,05. Применим критерий Манна-Уитни и выведем p-value. Также выведем относительное изменение конверсии группы B к группе А. Округлим до трёх знаков после запятой.

In [ ]:
print('Значение p-value: {0:.3f}'.format(stats.mannwhitneyu(sample_a, sample_b, True, 'two-sided')[1]))
print('Относительное изменение конверсии группы B к группе А: {0:.3f}'.format(sample_b.mean() / sample_a.mean() - 1))

Вывод: по «сырым» есть статистические различия в конверсии групп A и B.  
p-value = 0.017 меньше 0.05. Значит, нулевую гипотезу о том, что статистически значимых различий в конверсии между группами нет, отвергаем. Относительный прирост конверсии группы B равен 13.8%.

Эти данные данные также подтверждают график отностительного изменения кумулятивной конверсии, который показал прирост конверсии группы В к группе А к концу тестированая на уровне 14%.

### Cтатистическая значимость различий в среднем чеке заказа между группами по «сырым» данным

- Для того, чтобы определить каким критерием проверять статистическую значимость различий данных в выборках: параметрическим или непараметрическим, проверим данные на нормальное распределение. Для это используем критерий Шапиро-Уилка.

Сформулируем нулевую и альтернативную гипотезы:    
    
**H0** - суммы чеков пользователей распределены нормально    
**H1** - распределение не нормально
  
Установим уровень статистической значимости α = 0,05.

In [ ]:
alpha = 0.05  

results = stats.shapiro(orders[orders['group']=='A']['revenue'])
p_value = results[1]  

print('p-значение: ', p_value)

if p_value < alpha:
    print('Отвергаем нулевую гипотезу: распределение не нормально')
else:
    print('Не получилось отвергнуть нулевую гипотезу, всё нормально') 

In [ ]:
alpha = 0.05  

results = stats.shapiro(orders[orders['group']=='B']['revenue'])
p_value = results[1]  

print('p-значение: ', p_value)

if p_value < alpha:
    print('Отвергаем нулевую гипотезу: распределение не нормально')
else:
    print('Не получилось отвергнуть нулевую гипотезу, всё нормально') 

По итогам проверки выборок мы видим, что данные распределены не нормально, а значит для выявления различий нам стоит применить непараметрический критерий Манна-Уитни. 

Посчитаем статистическую значимость различий в среднем чеке между группами А/В-тестирвания по «сырым» данным.

Сформулируем нулевую и альтернативную гипотезы:    
    
**H0** - статистически значимых различий в среднем чеке между группами А/В-тестирования нет    
**H1** - средний чек между группами А/В-тестирования имеют статистически значимые отличия

Установим уровень статистической значимости α = 0,05. Применим критерий Манна-Уитни и выведем p-value. Также выведем относительное изменение среднего чека группы B к группе А. Округлим до трёх знаков после запятой.

In [ ]:
print('Значение p-value: {0:.3f}'.format(
    stats.mannwhitneyu(orders[orders['group']=='A']['revenue'], orders[orders['group']=='B']['revenue'], True, 'two-sided')[1]
))
print('Относительное изменение среднего чека группы B к группе А: {0:.3f}'.format(
    orders[orders['group']=='B']['revenue'].mean()/orders[orders['group']=='A']['revenue'].mean()-1
)) 

Вывод: По «сырым» данным различий в среднем чеке групп A и B нет.  
P-value значительно больше 0.05. Значит, причин отвергать нулевую гипотезу и считать, что в среднем чеке есть различия, нет. Средний чек группы B выше среднего чека группы A на 25,9%.

### Cтатистическая значимость различий в конверсии между группами по «очищенным» данным

Сделаем срезы пользователей с числом заказов больше 2 и пользователей, совершивших заказы дороже 30 000. Объединим их в таблице abnormal_users.

In [ ]:
# срез пользователей с числом заказов больше 2
users_with_many_orders = pd.concat(
    [
        orders_by_users_a[orders_by_users_a['orders'] > 2]['visitor_id'],
        orders_by_users_b[orders_by_users_b['orders'] > 2]['visitor_id'],
    ],
    axis=0,
)

# срез пользователей, совершивших заказы дороже 30 000
users_with_expensive_orders = orders[orders['revenue'] > 30000]['visitor_id']

# объединение таблиц
abnormal_users = (
    pd.concat([users_with_many_orders, users_with_expensive_orders], axis=0)
    .drop_duplicates()
    .sort_values()
)

print('Количество аномальных пользователей:', abnormal_users.shape)

Посчитаем статистическую значимость различий в конверсии между группами теста по очищенным данным. Сначала подготовим выборки количества заказов по пользователям по группам теста:

In [ ]:
sample_a_filtered = pd.concat(
    [
        orders_by_users_a[
            np.logical_not(orders_by_users_a['visitor_id'].isin(abnormal_users))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                total_data['visitors_per_date_a'].sum() - len(orders_by_users_a['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

sample_b_filtered = pd.concat(
    [
        orders_by_users_b[
            np.logical_not(orders_by_users_b['visitor_id'].isin(abnormal_users))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                total_data['visitors_per_date_b'].sum() - len(orders_by_users_b['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
) 

Сформулируем нулевую и альтернативную гипотезы:      
      
**H0** - статистически значимых различий в конверсии между группами А/В-тестирования нет      
**H1** - конверсии между группами А/В-тестирования имеют статистически значимые отличия  
  
Установим уровень статистической значимости α = 0,05. Применим критерий Манна-Уитни к полученным выбокам. Выведем относительное изменение конверсии группы B к группе А.

In [ ]:
print('Значение p-value: {0:.3f}'.format(stats.mannwhitneyu(sample_a_filtered, sample_b_filtered, True, 'two-sided')[1]))
print('Относительное изменение конверсии группы B к группе А: {0:.3f}'.format(
    sample_b_filtered.mean() / sample_a_filtered.mean() - 1
))

P-value практически не изменился, а относительный прирост конверсии группы В к группе А увеличился с 13,8% до 17,2%.  
Р-value остался также меньше 0.05, значит, нулевую гипотезу о том, что статистически значимых различий в конверсии между группами нет, отвергаем. 

### Cтатистическая значимость различий в среднем чеке заказа между группами по «очищенным» данным

Сформулируем нулевую и альтернативную гипотезы:      
      
**H0** - статистически значимых различий в среднем чеке между группами А/В-тестирования нет      
**H1** - средний чек между группами А/В-тестирования имеют статистически значимые отличия  
  
Установим уровень статистической значимости α = 0,05. Применим критерий Манна-Уитни к полученным данным и выведем относительное изменение среднего чека группы B к группе А.

In [ ]:
print(
    'Значение p-value: {0:.3f}'.format(
        stats.mannwhitneyu(
            orders[
                np.logical_and(
                    orders['group'] == 'A',
                    np.logical_not(orders['visitor_id'].isin(abnormal_users)),
                )
            ]['revenue'],
            orders[
                np.logical_and(
                    orders['group'] == 'B',
                    np.logical_not(orders['visitor_id'].isin(abnormal_users)),
                )
            ]['revenue'], True, 'two-sided'
        )[1]
    )
)

print(
    'Относительное изменение среднего чека группы B к группе А: {0:.3f}'.format(
        orders[
            np.logical_and(
                orders['group'] == 'B',
                np.logical_not(orders['visitor_id'].isin(abnormal_users)),
            )
        ]['revenue'].mean()
        / orders[
            np.logical_and(
                orders['group'] == 'A',
                np.logical_not(orders['visitor_id'].isin(abnormal_users)),
            )
        ]['revenue'].mean()
        - 1
    )
) 

P-value практически еще больше увеличился, но вот и разница между сегментами существенно изменилась. Средний чек группы B снизился относительно среднего чека группы A на с 25,9% до -2,3%.  
Не смотря на это, р-value остался значительно больше 0.05. Значит, причин отвергать нулевую гипотезу и считать, что в среднем чеке есть различия, нет. 

### Выводы и решения по результатам теста

На данный момент по результатам теста можно сделать следующие выводы:
- Есть статистически значимое различие по конверсии между группами как по «сырым», так и по данным после фильтрации аномалий;
- Нет статистически значимого различия по среднему чеку между группами ни по «сырым», ни по данным после фильтрации аномалий;
- График относительного различия конверсии между группами сообщает, что результаты группы B лучше группы A на 13,8% по «сырым» данным и на 17,2% по данным без аномалий;
- Средний чек группы B выше относительно среднего чека группы A по «сырым» данным, но наоборот ниже по данным без аномалий. Вероятно средний чек группы В еще не установился.

**Стоит продолжить тестирование, нужно больше данных, чтобы зафиксировался средний чек группы В. Группа В показывает хорошие результаты по конверсии, возможно средний срек группы В также установится выше среднего чека группы А, либо они не будут отличаться.**